In [1]:
pip install shap


   ---------------------------------------- 0.0/545.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/545.1 kB ? eta -:--:--
   ---------------------------------------- 545.1/545.1 kB 3.6 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   ---------------------------------------- 2/2 [shap]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import shap
import joblib
import pandas as pd

# Load your binary pipeline
pipe = joblib.load("best_model_pipeline.joblib")

# Create a small explainer using the trained RandomForest
rf = pipe.named_steps["model"]
prep = pipe.named_steps["prep"]

def explain_row(df_row):
    # preprocess to same format
    X_trans = prep.transform(df_row)
    explainer = shap.TreeExplainer(rf)
    shap_values = explainer.shap_values(X_trans)[1]  # class 1 = fail
    shap_abs = pd.Series(shap_values[0], index=prep.get_feature_names_out())
    return shap_abs.sort_values(ascending=False)


In [3]:
def rule_based_reason(row):
    reasons = []
    temp_diff = row["Process temperature [K]"].values[0] - row["Air temperature [K]"].values[0]
    if temp_diff > 80:
        reasons.append("High process–air temperature difference (possible heat-dissipation issue)")
    if row["Tool wear [min]"].values[0] > 200:
        reasons.append("Excessive tool wear")
    if row["Torque [Nm]"].values[0] > 60:
        reasons.append("High torque / overstrain")
    if row["Rotational speed [rpm]"].values[0] > 2000:
        reasons.append("High rotational speed")
    return reasons or ["No specific rule triggered"]
